<a href="https://colab.research.google.com/github/durgesh4802/PyTorch-for-Deep-Learning-and-Computer-Vision-Course-All-Codes-/blob/master/Credit_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the libraries used for machine learning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, accuracy_score,confusion_matrix,PrecisionRecallDisplay,roc_curve,roc_auc_score

import torch
from torch  import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/My Drive/Colab Notebooks/data/default_of_credit_card_clients.xls'
data=pd.read_excel(data_path)

In [4]:
data = data.iloc[1:,1:]
# YOUR CODE HERE
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
data['X2'],data['X3'],data['X4'] = data['X2'].astype('category'),data['X3'].astype('category'),data['X4'].astype('category')
data['X6'],data['X7'],data['X8'],data['X9'],data['X10'],data['X11'] = data['X6'].astype('category'),data['X7'].astype('category'),data['X8'].astype('category'),data['X9'].astype('category'),data['X10'].astype('category'),data['X11'].astype('category')
data['X1'],data['X5'],data['X12'],data['X13'],data['X14'],data['X15'],data['X16'],data['X17'],data['X18'],data['X19'],data['X20'],data['X21'],data['X22'],data['X23']= data['X1'].astype(float),data['X5'].astype(float),data['X12'].astype(float),data['X13'].astype(float),data['X14'].astype(float),data['X15'].astype(float),data['X16'].astype(float),data['X17'].astype(float),data['X18'].astype(float),data['X19'].astype(float),data['X20'].astype(float),data['X21'].astype(float),data['X22'].astype(float),data['X23'].astype(float)
data.info()
data.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 1 to 30000
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   X1      30000 non-null  float64 
 1   X2      30000 non-null  category
 2   X3      30000 non-null  category
 3   X4      30000 non-null  category
 4   X5      30000 non-null  float64 
 5   X6      30000 non-null  category
 6   X7      30000 non-null  category
 7   X8      30000 non-null  category
 8   X9      30000 non-null  category
 9   X10     30000 non-null  category
 10  X11     30000 non-null  category
 11  X12     30000 non-null  float64 
 12  X13     30000 non-null  float64 
 13  X14     30000 non-null  float64 
 14  X15     30000 non-null  float64 
 15  X16     30000 non-null  float64 
 16  X17     30000 non-null  float64 
 17  X18     30000 non-null  float64 
 18  X19     30000 non-null  float64 
 19  X20     30000 non-null  float64 
 20  X21     30000 non-null  float64 
 21  X22     3000

,count,mean,std,min,25%,50%,75%,max
X1,30000.0,167484.322667,129747.661567,10000.0,50000.00,140000.0,240000.00,1000000.0
X5,30000.0,35.485500,9.217904,21.0,28.00,34.0,41.00,79.0
X12,30000.0,51223.330900,73635.860576,-165580.0,3558.75,22381.5,67091.00,964511.0
X13,30000.0,49179.075167,71173.768783,-69777.0,2984.75,21200.0,64006.25,983931.0
X14,30000.0,47013.154800,69349.387427,-157264.0,2666.25,20088.5,60164.75,1664089.0
X15,30000.0,43262.948967,64332.856134,-170000.0,2326.75,19052.0,54506.00,891586.0
X16,30000.0,40311.400967,60797.155770,-81334.0,1763.00,18104.5,50190.50,927171.0
X17,30000.0,38871.760400,59554.107537,-339603.0,1256.00,17071.0,49198.25,961664.0
X18,30000.0,5663.580500,16563.280354,0.0,1000.00,2100.0,5006.00,873552.0
X19,30000.0,5921.163500,23040.870402,0.0,833.00,2009.0,5000.00,1684259.0


In [6]:
def preprocess_split(df):

    # Split df into X and y
    y = df['Y'].copy()
    y=y.astype(float)
    y=torch.tensor(y.values)
    y=y.view(-1,1)
    X = df.drop('Y', axis=1).copy()
    X=torch.tensor(X.values)
    # Scale X with a standard scaler
    # YOUR CODE HERE

    return X, y

In [7]:
X,y=preprocess_split(data)
print(f'type of x: {X.dtype} size of X:  {X.shape}  type of y:  {y.dtype} size of y:  {y.shape}')

type of x: torch.float64 size of X:  torch.Size([30000, 23])  type of y:  torch.float64 size of y:  torch.Size([30000, 1])


In [8]:
class Classifier(nn.Module):

    def __init__(self, D_in, H1, H2, D_out):
        super().__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, D_out)
    def forward(self, x):
        x = F.sigmoid(self.linear1(x))
        x = F.sigmoid(self.linear2(x))
        x = F.sigmoid(self.linear3(x))
        return x
    def predict(self, x):
        pred = self.forward(x)
        if pred >= 0.8:
          return 1
        else:
          return 0

In [9]:
torch.manual_seed(8)
model = Classifier(23, 6, 4, 1)
model

Classifier(
  (linear1): Linear(in_features=23, out_features=6, bias=True)
  (linear2): Linear(in_features=6, out_features=4, bias=True)
  (linear3): Linear(in_features=4, out_features=1, bias=True)
)

In [10]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001)

In [11]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 4.0838e-02,  1.4400e-01,  1.8617e-01, -8.4853e-02,  5.7558e-03,
          6.0196e-02,  1.6642e-01, -2.0264e-01,  3.2717e-02, -1.5772e-01,
          1.7436e-01,  7.5288e-02, -1.2510e-01, -1.6176e-01, -2.0770e-01,
          1.6382e-01,  1.2707e-01, -1.0231e-01, -1.3220e-02, -1.3836e-01,
         -1.3710e-01,  4.3031e-03, -1.6397e-01],
        [ 3.8011e-02,  1.2063e-01, -1.7290e-01, -5.5093e-02, -9.9710e-03,
          5.7807e-02, -2.1678e-02, -1.7536e-01, -7.5119e-02, -2.0718e-01,
         -3.1793e-02, -8.0511e-02,  1.8886e-01, -2.2455e-02,  1.1492e-01,
         -2.0306e-01, -3.7471e-02,  1.4469e-01,  1.2030e-01,  4.7900e-02,
         -1.2963e-01,  9.6360e-02,  1.9109e-01],
        [-1.4394e-02,  1.1340e-01, -1.4122e-01,  8.2769e-02,  1.7142e-02,
          1.8073e-01,  1.1282e-01,  1.9692e-01, -3.6231e-02, -4.7081e-03,
         -2.5784e-02, -2.7422e-02, -1.6854e-01, -1.4298e-01, -6.0300e-02,
         -1.4371e-01, -1.9546e-01, -9.9916e-02,  1.0624e-02, -2.0

In [12]:
epochs = 1000
losses = []

for i in range(epochs):
  y_pred = model.forward(X)
  print(f'size of X:  {y_pred.shape}   size of y:  {y.shape}')

  loss = criterion(y_pred, y)
  print("epoch:", i, "loss:", loss.item())
  losses.append(loss.item())
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


RuntimeError: ignored

In [ ]:
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.grid()

In [ ]:
# Plot the confusion matrix in the  subplots
import textwrap
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=plt.cm.summer, axes=None):
    if axes is None:
        axes = plt.gca()
    im = axes.imshow(cm, interpolation='nearest', cmap=cmap)
    #axes.imshow(cm, interpolation='nearest', cmap=cmap)
    axes.set_title(title)
    #plt.colorbar(ax=axes)
    tick_marks = np.arange(len(target_names))
    axes.set_xticks(tick_marks)
    axes.set_yticks(tick_marks)
    axes.set_xticklabels(target_names, rotation=45)
    axes.set_yticklabels(target_names)
    axes.set_xlabel('Predicted label')
    axes.set_ylabel('True label')
    axes.grid(False)
    fig.colorbar(im, ax=axes)
    width, height = cm.shape

    for x in range(width):
        for y in range(height):
            axes.annotate(str(cm[x][y]), xy=(y, x),
                        horizontalalignment='center',
                        verticalalignment='center', color='black', fontsize=22)

def plot_classification_report(cm,  title='Confusion matrix', cmap=plt.cm.summer, axes=None):
    axes.text(0.8, 0.5, cm, ha='right', va='center', transform=axes.transAxes)
    axes.set_axis_off()  # Turn off axes
    wrapped_title = '\n'.join(textwrap.wrap(title, width=26))
    axes.set_title(wrapped_title)
    border_props = dict(color='black', linewidth=2, fill=False)
    border_rect = plt.Rectangle((0, 0), 1, 1, transform=axes.transAxes, **border_props)
    axes.add_artist(border_rect)

In [ ]:
y_predict = model.predict(X)
target_names = ['0', '1']  # Replace with your class names
cnf_matrix = confusion_matrix(y, y_predict)
plot_confusion_matrix(cnf_matrix, target_names, title=f'Confusion Matrix Random Forests Accuracy: {accuracy_score(y, y_predict):.2f}')
